In [1]:
import pandas as pd
import geopandas as gpd
import zipfile
import numpy as np
import plotly.express as px

In [2]:
relations = [
    { "type": "subway", "from": "Odeonsplatz", "to": "Universität", "total": 110400 },
    { "type": "subway", "from": "Universität", "to": "Odeonsplatz", "total": 108100 },
    { "type": "subway", "from": "München Hbf", "to": "Stiglmaierplatz", "total": 48500 },
    { "type": "subway", "from": "Stiglmaierplatz", "to": "München Hbf", "total": 48500 },
    { "type": "subway", "from": "München Hbf", "to": "Königsplatz", "total": 74500 },
    { "type": "subway", "from": "Königsplatz", "to": "München Hbf", "total": 76900 },
    { "type": "subway", "from": "Theresienwiese", "to": "Schwanthalerhöhe", "total": 44500 },
    { "type": "subway", "from": "Schwanthalerhöhe", "to": "Theresienwiese", "total": 46300 },
    { "type": "subway", "from": "Sendlinger Tor", "to": "Goetheplatz", "total": 104500 },
    { "type": "subway", "from": "Goetheplatz", "to": "Sendlinger Tor", "total": 106500 },
    { "type": "subway", "from": "Sendlinger Tor", "to": "Fraunhoferstraße", "total": 89000 },
    { "type": "subway", "from": "Fraunhoferstraße", "to": "Sendlinger Tor", "total": 90500 },
    { "type": "subway", "from": "Max-Weber-Platz", "to": "Ostbahnhof", "total": 50600 },
    { "type": "subway", "from": "Ostbahnhof", "to": "Max-Weber-Platz", "total": 53500 },
    { "type": "subway", "from": "Kolumbusplatz", "to": "Silberhornstraße", "total": 62800 },
    { "type": "subway", "from": "Silberhornstraße", "to": "Kolumbusplatz", "total": 59300 },

    { "type": "tram", "from": "Karlstraße", "to": "Hauptbahnhof Nord", "total": 17300 },
    { "type": "tram", "from": "Hauptbahnhof Nord", "to": "Karlstraße", "total": 17200 },
    { "type": "tram", "from": "Hopfenstraße", "to": "Hauptbahnhof Nord", "total": 15100 },
    { "type": "tram", "from": "Hauptbahnhof Nord", "to": "Hopfenstraße", "total": 12400 },
    { "type": "tram", "from": "Holzkirchner Bahnhof", "to": "Hauptbahnhof Süd", "total": 12600 },
    { "type": "tram", "from": "Hauptbahnhof Süd", "to": "Holzkirchner Bahnhof", "total": 16400 },
    { "type": "tram", "from": "Karolinenplatz", "to": "Ottostraße", "total": 10000 },
    { "type": "tram", "from": "Ottostraße", "to": "Karolinenplatz", "total": 9800 },
    { "type": "tram", "from": "Müllerstraße", "to": "Sendlinger Tor", "total": 11000 },
    { "type": "tram", "from": "Sendlinger Tor", "to": "Müllerstraße", "total": 10800 },
]

### Load GTFS

In [3]:
df_zone = gpd.read_file("../../analysis/data/mvv_area.gpkg")

In [4]:
with zipfile.ZipFile("../../data/gtfs_idf/latest.zip") as archive:
    with archive.open("stops.txt") as f:
        df_stops = pd.read_csv(f, sep = ",")

df_stops["geometry"] = gpd.points_from_xy(df_stops["stop_lon"], df_stops["stop_lat"])
df_stops = gpd.GeoDataFrame(df_stops, crs = "EPSG:4326").to_crs("EPSG:25832")
df_stops = gpd.sjoin(df_stops, df_zone, predicate = "within")

### Find stop identifiers

In [5]:
names = set([r["from"] for r in relations] + [r["to"] for r in relations])
mapping = {} 

for name in names:
    f = df_stops["stop_name"] == name
    f &= ~df_stops["parent_station"].isna()

    if np.count_nonzero(f) == 0:
        print("No match for", name)

    else:
        parents = df_stops.loc[f, "parent_station"].unique()

        if len(parents) > 1:
            print("Multiple parents for", name, ":", parents)
        else:
            mapping[name] = parents[0]

In [6]:
for relation in relations:
    relation["from_id"] = mapping[relation["from"]]
    relation["to_id"] = mapping[relation["to"]]

### Load leg data

In [7]:
df = pd.read_csv("../../output/zones/eqasim_pt.csv", sep = ";")
sampling_rate = 0.01

In [8]:
df_comparison = []

for relation in relations:
    f = df["access_area_id"] == float(relation["from_id"])
    f &= df["egress_area_id"] == float(relation["to_id"])
    f &= df["transit_mode"] == relation["type"]

    simulation = np.count_nonzero(f) / sampling_rate
    
    df_comparison.append({
        "relation": "{} - {}".format(relation["from"], relation["to"]),
        "reference": relation["total"], "simulation": simulation, 
        "type": relation["type"]
    })

df_comparison = pd.DataFrame.from_records(df_comparison)

In [9]:
df_comparison

,relation,reference,simulation,type
0,Odeonsplatz - Universität,110400,0.0,subway
1,Universität - Odeonsplatz,108100,0.0,subway
2,München Hbf - Stiglmaierplatz,48500,0.0,subway
3,Stiglmaierplatz - München Hbf,48500,0.0,subway
4,München Hbf - Königsplatz,74500,0.0,subway
5,Königsplatz - München Hbf,76900,0.0,subway
6,Theresienwiese - Schwanthalerhöhe,44500,300.0,subway
7,Schwanthalerhöhe - Theresienwiese,46300,300.0,subway
8,Sendlinger Tor - Goetheplatz,104500,900.0,subway
9,Goetheplatz - Sendlinger Tor,106500,900.0,subway


In [10]:
px.scatter(df_comparison, x = "reference", y = "simulation", color = "type", trendline = "ols")